In [1]:
#Importing packages
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np

print("Libraries imported!")

Libraries imported!


In [2]:
r_cols = ['user_id', 'movie_id', 'rating']

ratings = pd.read_csv('data/ml-latest-small/ratings.csv')

m_cols = ['movie_id', 'title']
movies = pd.read_csv('data/ml-latest-small/movies.csv')

ratings_df = pd.merge(movies, ratings)

ratings_df = ratings_df[['movieId', 'title', 'userId', 'rating']]

print(ratings_df.shape)

ratings_df.head()

(100836, 4)


,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [3]:
ratings_df_coumns = ['movieId', 'title', 'userId', 'rating']

def add_fav_movies(title, userId, rating):
    
    movieId = ratings_df[ratings_df.title.str.contains(title)].movieId.values[0]
    
    title = ratings_df[ratings_df.title.str.contains(title)].title.values[0]
    
    ratings_df.loc[len(ratings_df)] = [movieId, title, userId, rating]
    
    return print(ratings_df.tail(5))

In [4]:
def run_rec_engine(min_periods, userId):
    
    user_ratings = ratings_df.pivot_table(index=['userId'], columns=['title'], values='rating')
    
    corr_matrix = user_ratings.corr(method='pearson', min_periods=min_periods)
    
    my_ratings = user_ratings.loc[userId].dropna()
    
    sim_candidates = pd.Series()

    for i in range(0, len(my_ratings.index)):
        
        print("Adding similarity for {} ...".format(my_ratings.index[i]))
        
        #Retrive similar movies for the one that I rated
        sims = corr_matrix[my_ratings.index[i]].dropna()
        
        #Scale its similarity by how well I rated this movie
        sims = sims.map(lambda x: x * my_ratings[i])
        
        #Add the score to the list of similarity candidates
        sim_candidates = sim_candidates.append(sims)
    print("----------------------")
    print("Based on your watching history, we found movies that you might like:")
    sim_candidates = sim_candidates.groupby(sim_candidates.index).sum()
    
    sim_candidates.sort_values(inplace=True, ascending=False)
    
    #filtered_candidates = sim_candidates.drop(my_ratings.index)
    
    return sim_candidates[len(my_ratings.index):len(my_ratings.index)+10]

In [5]:
add_fav_movies('Goodfellas', 771993, 5)
add_fav_movies('Pulp Fiction', 771993, 5)
add_fav_movies('Fight Club', 771993, 5)

        movieId                                title  userId  rating
100832   193583         No Game No Life: Zero (2017)     184     3.5
100833   193585                         Flint (2017)     184     3.5
100834   193587  Bungo Stray Dogs: Dead Apple (2018)     184     3.5
100835   193609  Andrew Dice Clay: Dice Rules (1991)     331     4.0
100836     1213                    Goodfellas (1990)  771993     5.0
        movieId                                title  userId  rating
100833   193585                         Flint (2017)     184     3.5
100834   193587  Bungo Stray Dogs: Dead Apple (2018)     184     3.5
100835   193609  Andrew Dice Clay: Dice Rules (1991)     331     4.0
100836     1213                    Goodfellas (1990)  771993     5.0
100837      296                  Pulp Fiction (1994)  771993     5.0
        movieId                                title  userId  rating
100834   193587  Bungo Stray Dogs: Dead Apple (2018)     184     3.5
100835   193609  Andrew Dice Clay:

In [6]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100839 entries, 0 to 100838
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   movieId  100839 non-null  int64  
 1   title    100839 non-null  object 
 2   userId   100839 non-null  int64  
 3   rating   100839 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 3.8+ MB


In [7]:
run_rec_engine(100, 771993)

Adding similarity for Fight Club (1999) ...
Adding similarity for Goodfellas (1990) ...
Adding similarity for Pulp Fiction (1994) ...
----------------------
Based on your watching history, we found movies that you might like:


Reservoir Dogs (1992)               4.280640
Godfather, The (1972)               4.049915
Seven (a.k.a. Se7en) (1995)         3.996019
Usual Suspects, The (1995)          3.852162
Memento (2000)                      3.805228
Matrix, The (1999)                  3.738990
Sixth Sense, The (1999)             3.091920
American Beauty (1999)              2.934422
Silence of the Lambs, The (1991)    2.829744
Saving Private Ryan (1998)          2.766417
dtype: float64

In [8]:
add_fav_movies('Notebook', 7794, 5)
add_fav_movies('Gone Girl', 7794, 5)
add_fav_movies('Help, The', 7794, 5)
run_rec_engine(20, 7794)

        movieId                                title  userId  rating
100835   193609  Andrew Dice Clay: Dice Rules (1991)     331     4.0
100836     1213                    Goodfellas (1990)  771993     5.0
100837      296                  Pulp Fiction (1994)  771993     5.0
100838     2959                    Fight Club (1999)  771993     5.0
100839     8533                 Notebook, The (2004)    7794     5.0
        movieId                 title  userId  rating
100836     1213     Goodfellas (1990)  771993     5.0
100837      296   Pulp Fiction (1994)  771993     5.0
100838     2959     Fight Club (1999)  771993     5.0
100839     8533  Notebook, The (2004)    7794     5.0
100840   112556      Gone Girl (2014)    7794     5.0
        movieId                 title  userId  rating
100837      296   Pulp Fiction (1994)  771993     5.0
100838     2959     Fight Club (1999)  771993     5.0
100839     8533  Notebook, The (2004)    7794     5.0
100840   112556      Gone Girl (2014)    7794 

Hangover, The (2009)                4.256693
Dark Knight, The (2008)             4.166958
Silence of the Lambs, The (1991)    3.851933
Inception (2010)                    3.737083
Up (2009)                           3.583080
Shrek (2001)                        3.561073
Shawshank Redemption, The (1994)    3.386408
Matrix, The (1999)                  3.290782
Beautiful Mind, A (2001)            3.253030
Titanic (1997)                      2.953113
dtype: float64

In [9]:
run_rec_engine(30, 7794)

Adding similarity for Gone Girl (2014) ...
Adding similarity for Help, The (2011) ...
Adding similarity for Notebook, The (2004) ...
----------------------
Based on your watching history, we found movies that you might like:


Inception (2010)                    2.245427
Shawshank Redemption, The (1994)    1.826625
Fight Club (1999)                   1.563248
dtype: float64